In [1]:
# imports
import sys
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import mdtraj

In [2]:
# load files
psf = app.CharmmPsfFile('init.psf')
pdbfile = app.PDBFile('init.pdb')
params = app.CharmmParameterSet('toppar/top_all36_prot.rtf', 'toppar/par_all36m_prot.prm', 'toppar/toppar_water_ions.str')
positions = pdbfile.getPositions(asNumpy=True)

In [3]:
# system
psf.setBox(39.0*unit.angstroms, 39.0*unit.angstroms, 39.0*unit.angstroms)

system = psf.createSystem(params,
    nonbondedMethod=app.PME, 
    nonbondedCutoff=1.2*unit.nanometers,
    implicitSolvent=None,
    constraints=app.HBonds,
    rigidWater=True, 
    verbose=True, 
    ewaldErrorTolerance=0.0005)

barostat = openmm.MonteCarloBarostat(1.0*unit.bar, 300.0*unit.kelvin)
system.addForce(barostat)

Adding particles...
Adding lonepairs...
Adding bonds...
Adding angles...
Adding Urey-Bradley terms
Adding torsions...
Adding impropers...
Adding CMAP coupled torsions...
Adding nonbonded interactions...


8

In [4]:
# integrator
integrator = openmm.LangevinIntegrator(300.0*unit.kelvin,   # Temperature of head bath
                                       1.0/unit.picosecond, # Friction coefficient
                                       0.002*unit.picoseconds) # Time step

In [5]:
# platform
platform = openmm.Platform.getPlatformByName('CPU')
# platform = openmm.Platform.getPlatformByName('CUDA')
# prop = dict(CudaPrecision='mixed')

In [6]:
# Build simulation context
simulation = app.Simulation(psf.topology, system, integrator, platform)
#simulation = app.Simulation(psf.topology, system, integrator, platform, prop)
simulation.context.setPositions(positions)

In [7]:
# initial system energy
print("\ninitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

initial system energy
-93500.13172582295 kJ/mol


In [8]:
# minimization
simulation.minimizeEnergy()

In [9]:
# energy after minimization
print("\nafter minimization")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

after minimization
-93032.7779830691 kJ/mol


In [10]:
# simulation
reportInterval = 10
nstep = 1000
simulation.reporters.append(app.StateDataReporter(sys.stdout, reportInterval, step=True, time=True, potentialEnergy=True, temperature=True, progress=True, remainingTime=True, speed=True, totalSteps=nstep, separator='\t'))
simulation.reporters.append(mdtraj.reporters.HDF5Reporter('run.h5', reportInterval, coordinates=True, time=True, cell=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(mdtraj.reporters.DCDReporter('run.dcd', reportInterval))
simulation.step(nstep)

#"Progress (%)"	"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	10	0.020000000000000004	-92831.67826723169	7.46687656174382	0	--


/opt/conda/lib/python3.7/site-packages/mdtraj/utils/validation.py:116: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  TypeCastPerformanceWarning)


2.0%	20	0.04000000000000002	-92569.17575958266	13.63722336101544	8.26	0:20
3.0%	30	0.06000000000000004	-92305.65828571701	19.47121697064734	6.08	0:27
4.0%	40	0.08000000000000006	-92045.90407593128	25.55769355845179	6.2	0:26
5.0%	50	0.10000000000000007	-91752.34134277604	30.234595764270615	5.78	0:28
6.0%	60	0.12000000000000009	-91507.63206555812	35.352376176715616	6.03	0:26
7.0%	70	0.1400000000000001	-91256.52640505886	40.396360671307804	6.21	0:25
8.0%	80	0.16000000000000011	-91078.40743223885	46.099517499240235	6.04	0:26
9.0%	90	0.18000000000000013	-90849.90812442376	50.72485400926056	6.16	0:25
10.0%	100	0.20000000000000015	-90616.34473607177	55.66503120874341	5.98	0:26
11.0%	110	0.22000000000000017	-90370.27524947969	59.988102528347234	6.11	0:25
12.0%	120	0.24000000000000019	-90185.21350657707	65.53766772443342	6.15	0:24
13.0%	130	0.2600000000000002	-89924.39771565591	69.7180779986045	6.08	0:24
14.0%	140	0.2800000000000002	-89750.33910846671	74.96347535062206	6.16	0:24
15.0%	150	0.300

In [11]:
# save checkpoint
with open('run.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())

In [12]:
del simulation # Make sure to close all files